<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs
!pip install jupyter-dash
!pip install dash-cytoscape

In [4]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json

In [5]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())
elements = data["nodes"] + data["edges"]
elements[:10]

[{'classes': 'inv', 'data': {'id': '2208', 'label': '0'}},
 {'classes': 'non_inv', 'data': {'id': '2207', 'label': '6'}},
 {'classes': 'non_inv', 'data': {'id': '2206', 'label': '6'}},
 {'classes': 'inv', 'data': {'id': '2204', 'label': '6'}},
 {'classes': 'inv', 'data': {'id': '2202', 'label': '6'}},
 {'classes': 'non_inv', 'data': {'id': '2198', 'label': '6'}},
 {'classes': 'non_inv', 'data': {'id': '2197', 'label': '6'}},
 {'classes': 'inv', 'data': {'id': '2196', 'label': '0'}},
 {'classes': 'non_inv', 'data': {'id': '2199', 'label': '6'}},
 {'classes': 'inv', 'data': {'id': '2193', 'label': '6'}}]

In [6]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '800px', 'height': '1000px'},
                                      stylesheet = [
        # Group selectors
        {
            'selector': 'node',
            'style': {
                'content': 'data(label)'
            }
        },

        # Class selectors
        {
            'selector': '.inv',
            'style': {
                'background-color': 'red',
                'line-color': 'red'
            }
        },
        {
            'selector': '.non_inv',
            'style': {
                'background-color': 'blue',
                'line-color': 'blue'
            }
        }
    ]
                                      )
                       ])
app.run_server(mode='inline')

<IPython.core.display.Javascript object>